In [1]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [2]:
%load_ext autoreload
%autoreload
from IPython.display import clear_output

import os
os.environ['PYTHONHASHSEED'] = '0'
os.environ["NUMEXPR_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OMP_NUM_THREADS"] = "1"

import sys
import argparse
import subprocess
import pdb
import time
import random
import _pickle as cPickle
import matplotlib.pyplot as plt
import glob

%matplotlib inline

import numpy as np
import pandas as pd
import tensorflow as tf

from data_structure import get_batches
from hntm import HierarchicalNeuralTopicModel
from tree import get_descendant_idxs
from evaluation import validate, get_hierarchical_affinity, get_topic_specialization, print_topic_sample
from configure import get_config

# load data & set config

In [3]:
config = get_config(nb_name)

In [4]:
os.environ["CUDA_VISIBLE_DEVICES"] = config.gpu
np.random.seed(config.seed)
random.seed(config.seed)

In [5]:
instances_train, categories, word_to_idx, idx_to_word, embeddings = cPickle.load(open(config.path_data, 'rb'))
config.batch_size = len(instances_train)
train_batches = get_batches(instances_train, config.batch_size, iterator=False)
config.dim_bow = len(idx_to_word)
config.dim_emb = embeddings.shape[-1]

In [6]:
def debug(variable, sample_batch=None, sample=False):
    if sample_batch is None: sample_batch = train_batches[0][1]
    feed_dict = model.get_feed_dict(sample_batch, mode='eval')
    _variable = sess.run(variable, feed_dict=feed_dict)
    return _variable

# run

## initialize log

In [7]:
checkpoint = []
losses_train = []
ppls_train = []
ppl_min = np.inf
epoch = 0

log_df = pd.DataFrame(columns=pd.MultiIndex.from_tuples(
                    list(zip(*[['','','','TRAIN:','','','','','VALID:','','','','','TEST:','', 'SPEC:', '', '', 'HIER:', ''],
                            ['Time','Ep','Ct','LOSS','PPL','NLL','KL','REG','LOSS','PPL','NLL','KL','REG','LOSS','PPL', '1', '2', '3', 'CHILD', 'OTHER']]))))

cmd_rm = 'rm -r %s' % config.dir_model
res = subprocess.call(cmd_rm.split())
cmd_mk = 'mkdir %s' % config.dir_model
res = subprocess.call(cmd_mk.split())

def update_checkpoint(config, checkpoint, global_step):
    checkpoint.append(config.path_model + '-%i' % global_step)
    if len(checkpoint) > config.max_to_keep:
        path_model = checkpoint.pop(0) + '.*'
        for p in glob.glob(path_model):
            os.remove(p)
    cPickle.dump(checkpoint, open(config.path_checkpoint, 'wb'))

## initialize model

In [8]:
if 'sess' in globals(): sess.close()
model = HierarchicalNeuralTopicModel(config)
sess = tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1))
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver(max_to_keep=config.max_to_keep)
update_tree_flg = False

## train & validate model

In [9]:
config.n_epochs = 5000
config.log_period = 500

In [10]:
time_start = time.time()
while epoch < config.n_epochs:
    ct, batch = train_batches[0]
    
    feed_dict = model.get_feed_dict(batch)
    _, loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch, ppls_batch, global_step_log = \
    sess.run([model.opt, model.loss, model.topic_loss_recon, model.topic_loss_kl, model.topic_loss_reg, model.topic_ppls, tf.train.get_global_step()], feed_dict = feed_dict)

    losses_train += [[loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch]]
    ppls_train += list(ppls_batch)

    if global_step_log % config.log_period == 0:
        # validate
        loss_train, topic_loss_recon_train, topic_loss_kl_train, topic_loss_reg_train = np.mean(losses_train, 0)
        ppl_train = np.exp(np.mean(ppls_train))
        loss_dev, topic_loss_recon_dev, topic_loss_kl_dev, topic_loss_reg_dev, ppl_dev, probs_topic_dev = validate(sess, train_batches, model)

        # test
        if ppl_train < ppl_min:
            ppl_min = ppl_dev
            loss_test, _, _, _, ppl_test, _ = 0, 0, 0, 0, 0, 0
            saver.save(sess, config.path_model, global_step=global_step_log)
            cPickle.dump(config, open(config.path_config % global_step_log, 'wb'))
            update_checkpoint(config, checkpoint, global_step_log)

        # visualize topic
        topics_freq_indices = np.argsort(sess.run(model.topic_bow), 1)[:, ::-1][:, :config.n_freq]
        topics_freq_idxs = topics_freq_indices
        topic_freq_tokens = {topic_idx: [idx_to_word[idx] for idx in topic_freq_idxs] for topic_idx, topic_freq_idxs in zip(model.topic_idxs, topics_freq_idxs)}
        topic_prob_topic = {topic_idx: prob_topic for topic_idx, prob_topic in zip(model.topic_idxs, probs_topic_dev)}
        descendant_idxs = {parent_idx: get_descendant_idxs(model, parent_idx) for parent_idx in model.topic_idxs}
        recur_prob_topic = {parent_idx: np.sum([topic_prob_topic[child_idx] for child_idx in recur_child_idxs]) for parent_idx, recur_child_idxs in descendant_idxs.items()}

        depth_specs = get_topic_specialization(sess, model, instances_train)
        hierarchical_affinities = get_hierarchical_affinity(sess, model)

        # log
        clear_output()
        time_log = int(time.time() - time_start)
        log_series = pd.Series([time_log, epoch, ct, \
                '%.2f'%loss_train, '%.0f'%ppl_train, '%.2f'%topic_loss_recon_train, '%.2f'%topic_loss_kl_train, '%.2f'%topic_loss_reg_train, \
                '%.2f'%loss_dev, '%.0f'%ppl_dev, '%.2f'%topic_loss_recon_dev, '%.2f'%topic_loss_kl_dev, '%.2f'%topic_loss_reg_dev, \
                '%.2f'%loss_test, '%.0f'%ppl_test, \
                '%.2f'%depth_specs[1], '%.2f'%depth_specs[2], '%.2f'%depth_specs[3], \
                '%.2f'%hierarchical_affinities[0], '%.2f'%hierarchical_affinities[1]],
                index=log_df.columns)
        log_df.loc[global_step_log] = log_series
        display(log_df)
        cPickle.dump(log_df, open(os.path.join(config.path_log), 'wb'))
        print_topic_sample(sess, model, topic_prob_topic=topic_prob_topic, recur_prob_topic=recur_prob_topic, topic_freq_tokens=topic_freq_tokens)

#         # update tree
#         if not config.static:
#             config.tree_idxs, update_tree_flg = model.update_tree(topic_prob_topic, recur_prob_topic)
#             if update_tree_flg:
#                 print(config.tree_idxs)
#                 name_variables = {tensor.name: variable for tensor, variable in zip(tf.global_variables(), sess.run(tf.global_variables()))} # store paremeters
#                 if 'sess' in globals(): sess.close()
#                 model = HierarchicalNeuralTopicModel(config)
#                 sess = tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1))
#                 name_tensors = {tensor.name: tensor for tensor in tf.global_variables()}
#                 sess.run([name_tensors[name].assign(variable) for name, variable in name_variables.items()]) # restore parameters
#                 saver = tf.train.Saver(max_to_keep=1)

        time_start = time.time()

    epoch += 1

TRAIN:                                  VALID:       \
     Time    Ep Ct       LOSS  PPL        NLL     KL   REG       LOSS  PPL   
500    11   499  0  101889.61  778  101866.29  22.82  0.45  101056.06  492   
1000    7   999  0  100896.27  695  100876.81  19.06  0.42   99682.62  626   
1500    7  1499  0  100490.25  672  100472.77  17.44  0.40   99669.01  619   
2000    7  1999  0  100284.59  661  100267.61  16.52  0.40   99664.77  624   
2500    7  2499  0  100162.38  654  100145.70  15.90  0.39   99660.90  617   
3000    7  2999  0  100078.24  649  100063.09  15.42  0.39   99657.80  614   
3500    7  3499  0  100017.65  645  100004.27  15.05  0.38   99656.52  617   
4000    7  3999  0   99972.09  642   99960.19  14.73  0.38   99655.37  612   
4500    7  4499  0   99936.47  640   99925.73  14.46  0.38   99655.70  616   
5000    7  4999  0   99910.05  638   99900.24  14.26  0.38   99656.16  617   

                             TEST:     SPEC:             HIER:        
            NLL     KL   REG  LOSS PPL     1     2     3 CHILD OTHER  
500   101039.85  15.78  0.44  0.00   0  0.14  0.76  0.92  0.07  0.07  
1000   99667.42  14.82  0.38  0.00   0  0.12  0.70  0.90  0.05  0.05  
1500   99654.82  13.81  0.37  0.00   0  0.12  0.70  0.89  0.05  0.05  
2000   99650.99  13.41  0.37  0.00   0  0.12  0.69  0.89  0.06  0.06  
2500   99646.90  13.63  0.37  0.00   0  0.12  0.70  0.89  0.06  0.05  
3000   99645.30  12.13  0.37  0.00   0  0.12  0.69  0.89  0.06  0.06  
3500   99643.52  12.64  0.37  0.00   0  0.12  0.69  0.89  0.06  0.06  
4000   99642.92  12.08  0.37  0.00   0  0.12  0.69  0.89  0.06  0.06  
4500   99643.43  11.90  0.36  0.00   0  0.12  0.69  0.89  0.06  0.06  
5000   99644.23  11.56  0.36  0.00   0  0.12  0.69  0.89  0.06  0.06

0 R: 1.000 P: 0.467 binder sensor sensors electrodes hollow_fiber_membranes actuators electrode lithium-ion_batteries ultrafiltration ultrafiltration_membranes
   1 R: 0.493 P: 0.477 vascular_grafts binder electrodes electrode vascular_access hemodialysis_access mitral_valve_repair chordal_replacement cathode femoropopliteal_bypass
     11 R: 0.015 P: 0.015 building_applications building_sector leads electric_wire fuel_cell_applications etfe_applications insulation building_materials etfe_cushion_models cables
     12 R: 0.000 P: 0.000 aerospace_applications pctfe-containing_ground_support_equipment representative_pctfe_replacement_parts "_at-risk_"_systems "_at-risk_"_high-pressure_oxygen air_systems stripline_resonator_measurement hot-melt_adhesives electronic_packaging_industry dielectric_layers
   2 R: 0.000 P: 0.000 lubricants greases lubricant lubrication ball_bearings tissue_engineering tribological_applications grease fuel_cells aerospace_industry
     21 R: 0.000 P: 0.000 spac